# CA 4 - Part 2 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 64.0 MB/s 
     |████████████████████████████████| 6.6 MB 57.6 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel, BertTokenizer
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

## Loading the data


In [5]:
train_df = pd.read_excel('/content/drive/MyDrive/Sheyda/NLP_Data/train.xlsx')
test_df = pd.read_excel('/content/drive/MyDrive/Sheyda/NLP_Data/test.xlsx')
valid_df = pd.read_excel('/content/drive/MyDrive/Sheyda/NLP_Data/valid.xlsx')

In [6]:
train_df.head()

,source,targets,category
0,"When news is brought to one of them, of (the b...",و چون یکی از آنان را به [ولادت] دختر مژده دهند...,quran
1,After them repaired Zadok the son of Immer ove...,و چون دشمنان ما شنیدند که ما آگاه شده‌ایم و خد...,bible
2,And establish regular prayers at the two ends ...,و نماز را در دو طرف روز و ساعات نخستین شب برپا...,quran
3,"And it came to pass, that, when I was come aga...",و فرمود تا مدعیانش نزد تو حاضر شوند؛ و از او ب...,bible
4,"Ah woe, that Day, to the Rejecters of Truth!",وای در آن روز بر تکذیب کنندگان!,quran


In [7]:
#Missing values in training set
train_df.isnull().sum()
#Missing values in test set
test_df.isnull().sum()

source      0
targets     0
category    0
dtype: int64

In [8]:
train_df.category.value_counts()

quran    4200
bible    4200
mizan    4200
Name: category, dtype: int64

In [9]:
print("Quran example :",train_df[train_df['category']=='quran']['source'].values[0], train_df[train_df['category']=='quran']['targets'].values[0])
print("Bible example :",train_df[train_df['category']=='bible']['source'].values[0], train_df[train_df['category']=='bible']['targets'].values[0])
print("Mizan example :",train_df[train_df['category']=='mizan']['source'].values[0], train_df[train_df['category']=='mizan']['targets'].values[0])

Quran example : When news is brought to one of them, of (the birth of) a female (child), his face darkens, and he is filled with inward grief! و چون یکی از آنان را به [ولادت] دختر مژده دهند [از شدت خشم] چهره‌اش سیاه گردد، ودرونش از غصه واندوه لبریز و آکنده شود!!
Bible example : After them repaired Zadok the son of Immer over against his house. After him repaired also Shemaiah the son of Shechaniah, the keeper of the east gate. و چون دشمنان ما شنیدند که ما آگاه شده‌ایم و خدا مشورت ایشان را باطل کرده است، آنگاه جمیع ما هر کس به‌کار خود به حصاربرگشتیم.
Mizan example : This man had become a just man in the full force of the term. این مرد، بت مام معنی کلمه، یک مرد درستکار شده بود.


In [10]:
train_df["category"] = train_df["category"].map({'mizan': 0, 'bible':1 , 'quran': 2})
test_df["category"] = test_df["category"].map({'mizan': 0, 'bible':1 , 'quran': 2})
valid_df["category"] = valid_df["category"].map({'mizan': 0, 'bible':1 , 'quran': 2})

## PART 1

In [11]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

EPOCHS = 10
EVERY_EPOCH = 500
LEARNING_RATE = 3e-5

In [18]:
X_train_e = train_df['source']
y_train_e = train_df['category']

X_test_fa = test_df['targets']
y_test_fa = test_df['category']

X_eval_e = valid_df['source']
y_eval_e = valid_df['category']

In [19]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = BertForSequenceClassification.from_pretrained(
    'xlm-roberta-base', 
    num_labels = 3
)

You are using a model of type xlm-roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.9.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.7.intermediate.dense.weight', 'roberta.encoder.layer.2.attention.self.key.weight', 'roberta.encoder.layer.9.output.LayerNorm.weight', 'roberta.encoder.layer.11.attention.output.dense.bias', 'roberta.encoder.layer.5.attention.output.dense.bias', 'roberta.encoder.layer.5.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.9.attention.self.value.weight', 'roberta.encoder.layer.2.intermediate.dense.bias', 'roberta.encoder.layer.10.attention.self.value.bias', 'roberta.encoder.layer.4.attention.self.value.weight', 'roberta.encoder.layer

In [20]:
X_train_e = X_train_e.values.tolist()
X_test_fa = X_test_fa.values.tolist()

y_train_e = y_train_e.values.tolist()
y_test_fa = y_test_fa.values.tolist()

X_eval_e = X_eval_e.values.tolist()
y_eval_e = y_eval_e.values.tolist()

In [21]:
train_encoding_e = tokenizer(X_train_e, padding=True, truncation=True, max_length=MAX_LEN, return_tensors = 'pt', return_attention_mask = True,)
test_encoding_e = tokenizer(X_test_fa, padding=True, truncation=True, max_length=MAX_LEN, return_tensors = 'pt', return_attention_mask = True,)
eval_encoding_e = tokenizer(X_eval_e, padding=True, truncation=True, max_length=MAX_LEN, return_tensors = 'pt', return_attention_mask = True,)

In [22]:
class SourcedetectionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
## Test Dataset
# class SourcedetectionTestDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         return item
#     def __len__(self):
#         return len(self.encodings)

## Generate Dataloaders

In [23]:
train_dataset = SourcedetectionDataset(train_encoding_e, y_train_e)
test_dataset = SourcedetectionDataset(test_encoding_e,y_test_fa)
val_dataset = SourcedetectionDataset(eval_encoding_e, y_eval_e)

In [24]:
test_dataset.encodings

{'input_ids': tensor([[     0,    498,  10196,  ...,      1,      1,      1],
        [     0,     65, 135218,  ...,      1,      1,      1],
        [     0,   1765, 119957,  ...,      1,      1,      1],
        ...,
        [     0,     65,  15675,  ...,      1,      1,      1],
        [     0,  22143,    870,  ...,      1,      1,      1],
        [     0,     65,   1228,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

## Define a Simple Metrics Function

In [26]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(labels, pred, average='weighted')

    return {"accuracy": accuracy,"f1_score":f1}

In [ ]:
training_args = TrainingArguments(
    output_dir='output',
    save_strategy='epoch',
    evaluation_strategy='epoch',
    eval_steps=EVERY_EPOCH,
    metric_for_best_model='accuracy',
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    load_best_model_at_end=True,
    save_total_limit = 1,
    logging_steps = 20

)
training_args

In [28]:
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [29]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12600
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3940
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.233800,0.198822,0.928889,0.928456
2,0.125900,0.145363,0.951852,0.951731
3,0.083500,0.160744,0.956667,0.956672
4,0.043600,0.182632,0.965185,0.965202
5,0.036000,0.211184,0.962222,0.962258
6,0.069900,0.195437,0.966296,0.966272
7,0.030100,0.224663,0.964444,0.964479
8,0.019000,0.233516,0.964815,0.964903
9,0.018500,0.236722,0.967037,0.967038
10,0.000100,0.241082,0.967037,0.967044


***** Running Evaluation *****
  Num examples = 2700
  Batch size = 32
Saving model checkpoint to output/checkpoint-394
Configuration saved in output/checkpoint-394/config.json
Model weights saved in output/checkpoint-394/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
***** Running Evaluation *****
  Num examples = 2700
  Batch size = 32
Saving model checkpoint to output/checkpoint-788
Configuration saved in output/checkpoint-788/config.json
Model weights saved in output/checkpoint-788/pytorch_model.bin
Deleting older checkpoint [output/checkpoint-394] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or 

TrainOutput(global_step=3940, training_loss=0.10013755348060152, metrics={'train_runtime': 1824.4286, 'train_samples_per_second': 69.063, 'train_steps_per_second': 2.16, 'total_flos': 8288072658432000.0, 'train_loss': 0.10013755348060152, 'epoch': 10.0})

In [30]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2700
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


{'epoch': 10.0,
 'eval_accuracy': 0.967037037037037,
 'eval_f1_score': 0.9670384888422484,
 'eval_loss': 0.23672173917293549,
 'eval_runtime': 10.1187,
 'eval_samples_per_second': 266.833,
 'eval_steps_per_second': 8.4}

In [31]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 2700
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


PredictionOutput(predictions=array([[ 6.294245 , -4.042062 , -3.2287474],
       [ 5.922205 , -4.5160885, -2.3854113],
       [ 5.90173  , -2.3339765, -4.5932097],
       ...,
       [ 6.316438 , -4.2778273, -3.0126653],
       [ 6.454144 , -3.911014 , -3.5302792],
       [ 6.1530776, -4.345872 , -2.770504 ]], dtype=float32), label_ids=array([0, 1, 1, ..., 2, 0, 1]), metrics={'test_loss': 6.341509819030762, 'test_accuracy': 0.33185185185185184, 'test_f1_score': 0.17940285904432687, 'test_runtime': 10.0753, 'test_samples_per_second': 267.983, 'test_steps_per_second': 8.436})

In [32]:
pin_memory=False
preds = trainer.predict(test_dataset=test_dataset)
preds

***** Running Prediction *****
  Num examples = 2700
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


PredictionOutput(predictions=array([[ 6.294245 , -4.042062 , -3.2287474],
       [ 5.922205 , -4.5160885, -2.3854113],
       [ 5.90173  , -2.3339765, -4.5932097],
       ...,
       [ 6.316438 , -4.2778273, -3.0126653],
       [ 6.454144 , -3.911014 , -3.5302792],
       [ 6.1530776, -4.345872 , -2.770504 ]], dtype=float32), label_ids=array([0, 1, 1, ..., 2, 0, 1]), metrics={'test_loss': 6.341509819030762, 'test_accuracy': 0.33185185185185184, 'test_f1_score': 0.17940285904432687, 'test_runtime': 10.0742, 'test_samples_per_second': 268.011, 'test_steps_per_second': 8.437})

In [33]:
preds.metrics

{'test_accuracy': 0.33185185185185184,
 'test_f1_score': 0.17940285904432687,
 'test_loss': 6.341509819030762,
 'test_runtime': 10.0742,
 'test_samples_per_second': 268.011,
 'test_steps_per_second': 8.437}

In [34]:
probs = torch.from_numpy(preds[0]).softmax(1)

# convert tensors to numpy array
predictions = probs.numpy()

In [35]:
predictions[0][0]/(predictions[0][0]+(0.5*(predictions[0][1] + predictions[1][0])))

0.6666973080985443

In [36]:
preds

PredictionOutput(predictions=array([[ 6.294245 , -4.042062 , -3.2287474],
       [ 5.922205 , -4.5160885, -2.3854113],
       [ 5.90173  , -2.3339765, -4.5932097],
       ...,
       [ 6.316438 , -4.2778273, -3.0126653],
       [ 6.454144 , -3.911014 , -3.5302792],
       [ 6.1530776, -4.345872 , -2.770504 ]], dtype=float32), label_ids=array([0, 1, 1, ..., 2, 0, 1]), metrics={'test_loss': 6.341509819030762, 'test_accuracy': 0.33185185185185184, 'test_f1_score': 0.17940285904432687, 'test_runtime': 10.0742, 'test_samples_per_second': 268.011, 'test_steps_per_second': 8.437})

In [37]:
newdf = pd.DataFrame(predictions,columns=['0','1','2'])

In [38]:
results = np.argmax(predictions,axis=1)
newdf['labels'] = results
newdf

,0,1,2,labels
0,0.999894,0.000032,0.000073,0
1,0.999724,0.000029,0.000247,0
2,0.999707,0.000265,0.000028,0
3,0.999882,0.000030,0.000088,0
4,0.999923,0.000030,0.000048,0
...,...,...,...,...
2695,0.999896,0.000063,0.000041,0
2696,0.999880,0.000024,0.000095,0
2697,0.999886,0.000025,0.000089,0
2698,0.999922,0.000032,0.000046,0


In [39]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
report = classification_report(y_test_fa, newdf['labels'], output_dict=True, zero_division=0)
report['auc_score'] = roc_auc_score(y_test_fa, probs, multi_class='ovr')

In [40]:
report

{'0': {'f1-score': 0.4978711325574794,
  'precision': 0.3343499809378574,
  'recall': 0.9744444444444444,
  'support': 900},
 '1': {'f1-score': 0.01756311745334797,
  'precision': 0.7272727272727273,
  'recall': 0.008888888888888889,
  'support': 900},
 '2': {'f1-score': 0.022774327122153208,
  'precision': 0.16666666666666666,
  'recall': 0.012222222222222223,
  'support': 900},
 'accuracy': 0.33185185185185184,
 'auc_score': 0.5771768518518519,
 'macro avg': {'f1-score': 0.17940285904432685,
  'precision': 0.4094297916257505,
  'recall': 0.33185185185185184,
  'support': 2700},
 'weighted avg': {'f1-score': 0.17940285904432687,
  'precision': 0.4094297916257505,
  'recall': 0.33185185185185184,
  'support': 2700}}